In [3]:
!pip install ja-ginza


In [9]:
import pandas as pd
import spacy
import itertools
from sklearn.feature_extraction.text import CountVectorizer
from typing import List, Tuple

# CSVファイルの読み込み
import pandas as pd
import spacy
import itertools
from sklearn.feature_extraction.text import CountVectorizer
from typing import List, Tuple

# CSVファイルの読み込み
data_path = './data/reviews_with_sentiment.csv'
df = pd.read_csv(data_path)

# spaCyモデルの読み込み
nlp = spacy.load('ja_ginza')

# 必要な品詞
POS = ['ADJ', 'ADV', 'INTJ', 'PROPN', 'NOUN', 'VERB']
MAX_TERMS_IN_DOC = 5
NGRAM = 1
MAX_DF = 1.0
MIN_DF = 0.0
NUM_VOCAB = 10000

# ヘルパー関数
def flatten(*lists) -> list:
    return list(itertools.chain.from_iterable(lists))

def remove_duplicates(l: List[Tuple[str, float]]) -> List[Tuple[str, float]]:
    return list({e[0]: e[1] for e in l}.items())

# 形態素解析の実施
df["doc"] = [nlp(review) for review in df["review"]]

# Bag-of-Wordsの作成
bows = {}
cvs = {}
for sentiment in df["sentiment"].unique():
    tokens = []
    for doc in df[df["sentiment"] == sentiment]["doc"]:
        similarities = [(token.similarity(doc), token.lemma_) for token in doc if token.pos_ in POS and token.has_vector]
        similarities = remove_duplicates(similarities)
        similarities = sorted(similarities, key=lambda sim: sim[0], reverse=True)[:MAX_TERMS_IN_DOC]
        tokens.append([similarity[1] for similarity in similarities])
    flattened_tokens = [' '.join(token_list) for token_list in tokens]
    cv = CountVectorizer(ngram_range=(1, NGRAM), max_df=MAX_DF, min_df=MIN_DF, max_features=NUM_VOCAB)
    bows[sentiment] = cv.fit_transform(flattened_tokens).toarray()
    cvs[sentiment] = cv
df = pd.read_csv(data_path)

# spaCyモデルの読み込み
nlp = spacy.load('ja_ginza')

# 必要な品詞
POS = ['ADJ', 'ADV', 'INTJ', 'PROPN', 'NOUN', 'VERB']
MAX_TERMS_IN_DOC = 5
NGRAM = 1
MAX_DF = 1.0
MIN_DF = 0.0
NUM_VOCAB = 10000

# ヘルパー関数
def flatten(*lists) -> list:
    return list(itertools.chain.from_iterable(lists))

def remove_duplicates(l: List[Tuple[str, float]]) -> List[Tuple[str, float]]:
    return list({e[0]: e[1] for e in l}.items())

# 形態素解析の実施
df["doc"] = [nlp(review) for review in df["review"]]

# Bag-of-Wordsの作成
bows = {}
cvs = {}
for sentiment in df["sentiment"].unique():
    tokens = []
    for doc in df[df["sentiment"] == sentiment]["doc"]:
        similarities = [(token.similarity(doc), token.lemma_) for token in doc if token.pos_ in POS and token.has_vector]
        similarities = remove_duplicates(similarities)
        similarities = sorted(similarities, key=lambda sim: sim[0], reverse=True)[:MAX_TERMS_IN_DOC]
        tokens.append([similarity[1] for similarity in similarities])
    flattened_tokens = [' '.join(token_list) for token_list in tokens]
    cv = CountVectorizer(ngram_range=(1, NGRAM), max_df=MAX_DF, min_df=MIN_DF, max_features=NUM_VOCAB)
    bows[sentiment] = cv.fit_transform(flattened_tokens).toarray()
    cvs[sentiment] = cv


In [ ]:
import numpy as np

TOP_K = 20

# 単語の登場頻度を計算
vocabs = {}
term_frequencies = {}
for sentiment in df["sentiment"].unique():
    bow = bows[sentiment]
    cv = cvs[sentiment]
    vocab = cv.vocabulary_
    term_frequency = np.sum(bow, axis=0)
    vocabs[sentiment] = vocab
    term_frequencies[sentiment] = term_frequency
    indices_topk = np.argsort(term_frequency)[::-1][:TOP_K]
    bow_topk = np.take(bow, indices_topk, axis=1)
    reverse_vocab = {vocab[k]: k for k in vocab.keys()}
    words = [reverse_vocab[i] for i in indices_topk]
    
    print(f"{sentiment} :")
    for w, c in zip(words, term_frequency[indices_topk]):
        print(w, ":", c)


In [ ]:
!pip install python-dp

In [ ]:

import numpy as np
from pydp.algorithms.laplacian import Count
from typing import List, Tuple


In [ ]:
def preprocess_for_private_counts(tf: np.ndarray) -> List[np.ndarray]:
    repeated_words = []
    for i, term in enumerate(tf):
        repeated_words.append(np.repeat(i, term))
    return repeated_words


In [ ]:
def cal_private_count(
    epsilon: float,
    max_partition_contributed: float,
    max_contributions_per_partition: float,
    repeated_words: List[np.ndarray],
) -> List[int]:
    private_counts = []
    for repeated_word in repeated_words:
        counter = Count(epsilon, max_partition_contributed, max_contributions_per_partition)
        count = counter.quick_result(repeated_word)
        private_counts.append(count)
    return private_counts

def top_k_words_and_counts(k: int, tf: np.ndarray, vocab: dict) -> List[Tuple[str, int]]:
    indices_topk = np.argsort(tf)[::-1][:k]
    reverse_vocab = {vocab[key]: key for key in vocab.keys()}
    words = [reverse_vocab[i] for i in indices_topk]
    counts = [tf[i] for i in indices_topk]
    return list(zip(words, counts))


In [ ]:
epsilons = [0.01, 0.05, 0.1, 0.3, 0.7, 1.0, 2.0, 3.0, 7.0, 10.0]
MAX_DUPLICATED_TERMS = 1

for eps in epsilons:
    print(f"ε: {eps}")
    for sentiment in df["sentiment"].unique():
        repeated_words = preprocess_for_private_counts(term_frequencies[sentiment])
        private_counts = cal_private_count(eps, MAX_TERMS_IN_DOC, MAX_DUPLICATED_TERMS, repeated_words)
        words_and_counts = top_k_words_and_counts(TOP_K, private_counts, vocabs[sentiment])
        print(f"{sentiment} : {words_and_counts}")